In [ ]:
import glob
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
# Load LLaMA Model

model_id = "meta-llama/Meta-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
# Load Documents from Path using TextLoader

pdf_files = glob.glob("./data/*.pdf")  
documents = []

for pdf_file in pdf_files:
    pdf_loader = PyPDFLoader(pdf_file)  
    documents.extend(pdf_loader.load())

In [ ]:
# Split large documents into chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(documents)

In [ ]:
# Embed the documents and initialize Chroma vector store

embedding_model = HuggingFaceEmbeddings(model_id="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Chroma(embedding_function=embedding_model, persist_directory="./db")
vector_store.add_documents(split_documents)
vector_store.persist()

In [ ]:
# Create a Retriever and QA Chain

retriever = vector_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=model, retriever=retriever)

In [ ]:
# Set Up a Simple Query Function to Run RAG

def ask_query(query):
    response = qa_chain.run(query)
    return response

In [ ]:
# Query the Model

query = "What is Hugging Face."
response = ask_query(query)

In [ ]:
print(response)